In [17]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameReader
from pyspark.sql.types import DoubleType
from pyspark.pandas import DataFrame
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext


In [34]:
properties = {"user": "pietro", "password": "ropby8Pietro", "driver": 'org.postgresql.Driver'}
url = "jdbc:postgresql://localhost:5432/dataimporta"
tablename="peru_imp"

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "postgresql-42.2.14.jar") \
    .getOrCreate()
    

peru = spark.read \
    .jdbc(url, tablename,
          properties=properties)
#peru.show(1)

In [ ]:
Sc = SparkContext()
sqlContext = SQLContext(sc)

In [36]:
peru.printSchema()
peru.describe().toPandas().transpose()

root
 |-- country_of_arrival: string (nullable = true)
 |-- mean_of_transport: string (nullable = true)
 |-- price_transport_net: string (nullable = true)
 |-- price_transport_net_insurance: string (nullable = true)
 |-- net_price_per_unit: string (nullable = true)
 |-- commercial_description: string (nullable = true)
 |-- custom: string (nullable = true)
 |-- date: string (nullable = true)
 |-- net_price: string (nullable = true)
 |-- country: string (nullable = true)
 |-- type: string (nullable = true)



,0,1,2,3,4
summary,count,mean,stddev,min,max
country_of_arrival,206581,None,None,ALBANIA,ZONAS FRANCAS DEL PERU
mean_of_transport,206581,None,None,AVION,VAPOR
price_transport_net,206581,5959.787971788306,225830.21328652062,0,9999
price_transport_net_insurance,206581,5973.86558299166,225874.4042029432,0,9999
net_price_per_unit,206581,8942804.224496804,2.823329292905949E8,0.0,9999.9950000025
commercial_description,206581,None,None,"""BRAZO RAM DOBLE TOMA 8"""" P/ BOLA1.,RAM,RAM-20...","Ã5MM AXIALER NEEDLEHOLDER,STEMA MEDIZINTECHNI..."
custom,206581,153.17184542624926,55.53537490336021,118,91
date,206581,2.022020922686501E7,1.5324005129741156,20220207,20220213
net_price,206581,5379.637134465546,217008.38531584118,0.01,99999.165


In [42]:
# SELECT THE COLUMNS FOR THE TASK
peru.createOrReplaceTempView("peruTW")
data = spark.sql("SELECT (price_transport_net-net_price) AS price_transport, mean_of_transport, country_of_arrival, custom, (net_price/net_price_per_unit) AS units, date, net_price  FROM peruTW")


In [43]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
price_transport,206581,580.1508373228568,11057.136028710764,-0.9989999999999997,2980141.9899999946
mean_of_transport,206581,None,None,AVION,VAPOR
country_of_arrival,206581,None,None,ALBANIA,ZONAS FRANCAS DEL PERU
custom,206581,153.17184542624926,55.53537490336021,118,91
units,205319,12719.207531192142,2571516.481089632,1.0E-6,8.65000000000001E8
date,206581,2.022020922686501E7,1.5324005129741156,20220207,20220213
net_price,206581,5379.637134465546,217008.38531584118,0.01,99999.165


In [44]:
import pandas as pd
numeric_features = ['units', 'net_price']
sampled_data = data.select(numeric_features).sample(False, 0.8).toPandas()
#axs = pd.scatter_matrix(sampled_data, figsize=(10, 10))
n = len(sampled_data.columns)


In [54]:
import six
for i in data.columns:
    if not( isinstance(data.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to price_transport for ", i, data.stat.corr('price_transport',i))

Correlation to price_transport for  price_transport 1.0
Correlation to price_transport for  units 0.04427172889464597


In [41]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['mean_of_transport', 'country_of_arrival', 'custom', 'units', 'date', 'net_price'], outputCol = 'price_transport')
tcompany_df = vectorAssembler.transform(data)
tcompany_df = tcompany_df.select(['features', 'Employees'])
tcompany_df.show(3)

IllegalArgumentException: Data type string of column mean_of_transport is not supported.
Data type string of column country_of_arrival is not supported.
Data type string of column custom is not supported.
Data type string of column date is not supported.
Data type string of column net_price is not supported.

In [38]:
splits = data.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [39]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='Emplyees', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

IllegalArgumentException: features does not exist. Available: price_transport, mean_of_transport, country_of_arrival, custom, units, date, net_price